In [1]:
import pymongo
from pymongo import MongoClient
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup

load_dotenv()

try:
    jumia_flash_sales_url  = 'https://www.jumia.co.ke/flash-sales/?page={}#catalog-listing'

    MONGODB_URI = os.getenv('MONGODB_URI')
    client = MongoClient(MONGODB_URI)
    # print(client)
    database = client["testPyDB"]
    collection = database["products"]
    # print(collection)

    page = 1
    while True:
        url = jumia_flash_sales_url.format(page)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to fetch page: {page}. Status Code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        products_list = soup.find_all("article", class_="prd _fb _p col c-prd")

        if not products_list:
            break

        for product in products_list:
            product_name = product.find("h3", class_="name").get_text()
            current_price = product.find("div", class_="prc").get_text()
            old_price_tag = product.find("div", class_="old")
            previous_price = old_price_tag.getText() if old_price_tag else "NaN"
            image_url = product.find("div", class_="img-c").find("img", class_="img").get('src')
            offical_store_tag = product.find("div", class_="bdg _mall _xs")
            from_official_store = True if offical_store_tag else False
            items_left = product.find("div", class_="stk").get_text()

            product_db_list = {
                    "Name" : product_name,
                    "Current Price" : current_price,
                    "Previous Price" : previous_price,
                    "Image URL" : image_url,
                    "IsFromOfficalStore" : from_official_store,
                    "Items Left" : items_left
                  }

            result = collection.insert_one(product_db_list)
            print(result.acknowledged)

        page += 1

    client.close()
except Exception as e:
    raise Exception("The following error occurred: ", e)

ModuleNotFoundError: No module named 'pymongo'